In [1]:
from itertools import chain
from pathlib import Path

from IPython.display import Markdown, display
from tqdm.notebook import tqdm

import hashlib
from os import environ
import pyperclip
import subprocess

from githelpers import open_pbcopy
import ghapicache
import release_notes

%load_ext autoreload
%autoreload 1
%aimport githelpers
%aimport ghapicache
%aimport release_notes

In [2]:
cached = ghapicache.GhApiCache()
ghapi = cached.api

In [3]:
# Load teams
TEAMS = {}
for t in tqdm(cached.teams()):
    TEAMS[t['name']] = release_notes.get_team(cached, t)

team_members = frozenset(chain(*(t.members for t in TEAMS.values())))
org_members = frozenset(m['login'] for m in cached.org_members())
TEAMS['affiliates'] = release_notes.Team(description="Associated with Celeritas but not core members",
                                         members=(org_members - team_members))

# Initialize the user cache
local_repo = Path("/Users/seth/Code/celeritas-temp")
user_cache = release_notes.UserCache(cached, local_repo / "scripts/release/users.json")
for login in tqdm(org_members):
    user_cache[login]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

# List active members

This is to be used for crediting in presentations, etc.

In [4]:
text = []
for team, title in [
    ('code-lead', 'Code lead'),
    ('core', 'Core members'),
    ('core-advisor', 'Core advisors'),
    ('affiliates', 'Affiliates'),]:
    text.append(f"## {title}")
    m = [user_cache[username] for username in TEAMS[team].members]
    for member in sorted(m, key=release_notes.get_last_name):
        text.append("- " + release_notes.format_user(member))
    text.append("")

display(Markdown("\n".join(text)))

## Code lead
- Seth R. Johnson *(ORNL)*

## Core members
- Elliott Biondo *(ORNL)*
- Julien Esseiva *(LBNL)*
- Hayden Hollenbeck *(UVA)*
- Seth R. Johnson *(ORNL)*
- Soon Yung Jun *(FNAL)*
- Guilherme Lima *(FNAL)*
- Amanda Lund *(ANL)*
- Ben Morgan *(U Warwick)*
- Sakib Rahman *(BNL)*
- Stefano Tognini *(ORNL)*

## Core advisors
- Philippe Canal *(FNAL)*
- Marcel Demarteau *(ORNL)*
- Tom Evans *(ORNL)*

## Affiliates
- Lance Bullerwell *(ORNL)*
- Wouter Deconinck *(U Manitoba)*
- Sam Eriksen *(U Bristol)*
- Steven Hamilton *(ORNL)*
- Paul Romano *(ANL)*
- Frederic Suter *(ORNL)*
- Sandro Wenzel *(CERN)*


## Release note generation

- Merge base should be all commits *already* released (skip documenting)
- Target branch is the one where the release candidate is
- Previous major branch allows all "v.x" contributors to be credited

In [5]:
ReleaseMetadata = release_notes.ReleaseMetadata

# All commits from all time
all_md = ReleaseMetadata(
    merge_bases=['v0.0.0'],
)

all_prs = release_notes.PullRequestRange(all_md)
count_contrib = release_notes.ContributionCounter(cached)
for pr in tqdm(all_prs.pull_ids):
    count_contrib(pr)
authors = count_contrib.sorted().author
authors

Can't match log subject to PR: Format code base (clang-format version 11.0.1)


  0%|          | 0/1415 [00:00<?, ?it/s]

{'sethrj': 841,
 'amandalund': 205,
 'esseivaju': 88,
 'stognini': 48,
 'whokion': 46,
 'pcanal': 45,
 'elliottbiondo': 33,
 'mrguilima': 31,
 'drbenmorgan': 22,
 'paulromano': 15,
 'hhollenb': 13,
 'vrpascuzzi': 7,
 'tmdelellis': 6,
 'VHLM2001': 4,
 'dalg24': 3,
 'lebuller': 3,
 'DoaaDeeb': 1,
 'aprokop': 1,
 'hartsw': 1,
 'ptheywood': 1,
 'rahmans1': 1}

In [6]:
print("Missing ORCIDs:")
print(" ".join(f"@{u}" for u in authors if user_cache[u].orcid is None))

Missing ORCIDs:
@VHLM2001 @DoaaDeeb


## Release

In [7]:
# Major release
major_md = ReleaseMetadata(
    release='0.6.0',
    merge_bases=['v0.5.0', 'v0.5.2'],
)

# Minor release
minor_md = ReleaseMetadata(
    release='0.5.3',
    merge_bases=['v0.5.2'],
    target_branch='backports/v0.5'
)

In [8]:
if 1:
    # Backport release:
    release_md = minor_md
    note_body =  """
Version {release} is a minor update to Celeritas featuring an important physics bugfix
to Urban MSC and additional minor fixes targeting CUDA VecGeom compatibility.
"""
else:
    # Major release
    release_md = major_md
    note_body = """
Version {release} is a major update to Celeritas featuring:

- 

A few minor features are noteworthy:

- 

Important changes:

- 

Notable bug fixes include:

- 

Some interfaces have been removed:

- 
"""

In [9]:

prs = release_notes.PullRequestRange(release_md)
sorted_pulls = release_notes.SortedPulls(cached)
count_contrib = release_notes.ContributionCounter(cached)
for pr_id in tqdm(prs.pull_ids):
    try:
        count_contrib(pr_id)
        sorted_pulls.add(pr_id)
    except Exception as e:
        print(f"Error adding PR #{pr_id}: {e}")

reviewers = count_contrib.sorted().reviewer
for login in tqdm(reviewers):
    user_cache[login]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
def make_notes(cls, release_md, note_body):
    prev = release_md.merge_bases[0]
    other = release_md.merge_bases[1:]
    if not other:
        change_str = f"Changes since {prev} follow."
    else:
        assert len(other) == 1
        change_str = f"Changes since {prev}, excluding those released in {other[0]}, follow."

    notes = cls(release_md, note_body)
    notes.paragraph(change_str)
    notes.sorted_pulls(sorted_pulls)
    notes.reviewers(reviewers, user_cache)
    notes.changelog_line("celeritas-project", "celeritas")

    return notes

In [11]:
rst_notes = make_notes(release_notes.RstNotes, release_md, note_body)

with open_pbcopy() as pb:
    rst_notes.write(pb)

# Draft github release

In [12]:
ghapi = cached.api

In [13]:
markdown_notes = make_notes(release_notes.MarkdownNotes, release_md, note_body)


In [ ]:
gh_release = release_notes.find_release(ghapi, release_md.release)
if not gh_release:
    gh_release = release_notes.create_release(ghapi, release_md, str(markdown_notes))

In [ ]:
artifact_url, artifact_tgz = release_notes.get_or_upload_tarball(cached, gh_release)
print(f"Artifact URL: {artifact_url}")

Loading https://api.github.com/repos/celeritas-project/celeritas/tarball/v0.5.3 from cached file data/ghapicache-downloads/443893a0ecbd8acfbcc3e97630efd70a32cbfa86.3
Uploading release tarball
Uploaded artifact: https://github.com/celeritas-project/celeritas/releases/download/v0.5.3/release-v0.5.3.tar.gz
Artifact URL: https://github.com/celeritas-project/celeritas/releases/download/v0.5.3/release-v0.5.3.tar.gz


In [25]:
#assert 0
sha256_hash = hashlib.sha256(artifact_tgz).hexdigest()
with open_pbcopy() as pb:
    pb.write(f'version("{release_md.release}", sha256="{sha256_hash}")\n')
print("Spack version copied to clipboard!")
subprocess.check_call(
    [
        "open",
        Path(environ["SPACK_ROOT"])
        / "var/spack/repos/builtin/packages/celeritas/package.py",
    ]
)

Spack version copied to clipboard!


0

# Push to zenodo

In [26]:
import zenodoapi
%aimport zenodoapi

In [27]:
# Load the Zenodo token
token_path = Path.home() / ".config/zenodo-token"
with open(token_path) as f:
    zenodo_token = f.read().strip()

zenodo = zenodoapi.Zenodo(zenodo_token)
zenodo.api_url = "https://zenodo.org/api/"

In [40]:
def load_contributions(release_md):
    prs = release_notes.PullRequestRange(release_md)
    count_contrib = release_notes.ContributionCounter(cached)
    for pr in tqdm(prs.pull_ids):
        count_contrib(pr)

    # Create author list
    return count_contrib.sorted()

def download_tarball(gh_release):
    asset = gh_release['assets'][0]
    name = asset['name']
    content = cached.download_file(asset['url'], ext=Path(name).suffix)
    assert content is not None
    return (content, name)

make_zenodo_md = release_notes.ZenodoMetadataBuilder(user_cache=user_cache, teams=TEAMS)

## Minor release

This assumes the major release has already been published upstream into Zenodo.

In [39]:
major, minor, patch = release_md.release.split('.')
release_md = release_notes.ReleaseMetadata.from_comprehensive_version(minor, patch)
major_dep = zenodo.find_deposition(f"Celeritas {major}.{minor}")

In [50]:
major_dep.links

{'self': 'https://zenodo.org/api/records/15177269',
 'html': 'https://zenodo.org/records/15177269',
 'doi': 'https://doi.org/10.5281/zenodo.15177269',
 'parent_doi': 'https://doi.org/10.5281/zenodo.15175891',
 'badge': 'https://zenodo.org/badge/doi/10.5281%2Fzenodo.15177269.svg',
 'conceptbadge': 'https://zenodo.org/badge/doi/10.5281%2Fzenodo.15175891.svg',
 'files': 'https://zenodo.org/api/records/15177269/files',
 'latest_draft': 'https://zenodo.org/api/deposit/depositions/15177269',
 'latest_draft_html': 'https://zenodo.org/deposit/15177269',
 'publish': 'https://zenodo.org/api/deposit/depositions/15177269/actions/publish',
 'edit': 'https://zenodo.org/api/deposit/depositions/15177269/actions/edit',
 'discard': 'https://zenodo.org/api/deposit/depositions/15177269/actions/discard',
 'newversion': 'https://zenodo.org/api/deposit/depositions/15177269/actions/newversion',
 'registerconceptdoi': 'https://zenodo.org/api/deposit/depositions/15177269/actions/registerconceptdoi',
 'record': 

In [47]:
zenodo.get_deposition(major_dep.get_latest_version().id)

ZenodoDeposition(id=15177269, title="Celeritas 0.5")

In [44]:
gh_release = release_notes.find_release(ghapi, release_md.release)

In [ ]:
# Get the Zenodo metadata    
from requests import HTTPError


old_md = major_dep.data["metadata"]
new_md = make_zenodo_md(load_contributions(release_md), release_md, gh_release)
# Only update editors, not team members
old_contrib = new_md["contributors"]
new_contrib = [u for u in old_contrib if u["type"] == "Editor"]
old_contrib = [u for u in old_contrib if u["type"] != "Editor"]
new_md["contributors"] = new_contrib + old_contrib
# Don't change the title
new_md["title"] = major_dep.data["metadata"]["title"]
# Add the body
new_md["description"] = "\n\n".join(
    [
        old_md["description"],
        f"<h1>Version {release_md.release}</h1>",
        new_md["description"],
    ]
)
try:
    new_vers = major_dep.create_new_version()
except HTTPError as e:
    # Maybe the version already exists
    if e.response.status_code == 400:
        print("Draft may already exist:", e)
        new_vers = major_dep.get_latest_draft()
    else:
        raise
try:
    new_vers.update(new_md)
except HTTPError as e:
    # Something else went wrong? Try to upload the tarball
        print("Draft may already exist:", e)

# Upload the release
(content, name) = download_tarball(gh_release)
new_vers.upload(content, name)
# The old tarball may still be there (this is buggy) so delete it
new_vers.refresh()
for file in new_vers.get_files():
    if file.filename != name:
        try:
            file.delete()
        except Exception as e:
            print(f"Failed to delete {file.filename}: {e}")
new_vers.refresh()

Can't match log subject to PR: Define BuildFlags dependency helper target
Can't match log subject to PR: Print cmake version
Can't match log subject to PR: Extracts from 582256d4f
Can't match log subject to PR: Update workflows from 245c2edea


  0%|          | 0/405 [00:00<?, ?it/s]

Draft may already exist


HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: https://zenodo.org/api/records/15177269?access_token=qyBwFLjLS4COh2W1KA8EzIKpOgpNC7LXTkTagCYi4QtQ5zaxYzaVNL6TZx0n